In [21]:
from advent import get_lines

data = get_lines(24)

In [22]:
# I will represent state as a list of 4 integers, since there are only 4 vars
# this is just the MONAD reengineered, but not really crucial to the actual solution, which starts next cell

state_ix = {'w': 0, 'x': 1, 'y': 2, 'z': 3}

def is_const(val):
    if isinstance(val, int): return True
    return all(v in '0123456789-' for v in val)

def get_value(state, var):
    if is_const(var):
        return int(var)
    val = state[state_ix[var]]
    return int(val) if is_const(val) else val

def set_value(state, var, value):
    # immutable. example: set_value([1, 0, 0, 1], 'y', 2) -> [1, 0, 2, 1]
    state_ = state.copy()
    state_[state_ix[var]] = value
    return state_

def apply_aritmetic(instr, a, b):
    # a and b must be integers. instr cannot be 'inp'
    if instr == 'add': return a + b
    if instr == 'mul': return a * b
    if instr == 'div': return int(a / b)
    if instr == 'mod': return a % b
    if instr == 'eql': return 1 if a == b else 0
    raise ValueError(f"{instr} is not a correct aritmetic operation")

# given an instruction, apply it to the state (and possibly the model number) and return a new state/modelnumber
def apply_instruction(instruction, state, model_number):
    if instruction.startswith('inp'):
        return set_value(state, instruction[-1], model_number[0]), model_number[1:]
    instr, var_a, var_b = instruction.split(' ')
    const_a, const_b = get_value(state, var_a), get_value(state, var_b)
    state = set_value(state, var_a, apply_aritmetic(instr, const_a, const_b))
    return state, model_number

def apply_instructions_list(instructions_list, model_number = [9]*14, state=[0, 0, 0, 0]):
    for instruction in instructions_list:
        state, model_number = apply_instruction(instruction, state, model_number)
    return state

apply_instructions_list(data)

[9, 1, 23, 5520630905]

In [23]:
# Plan: create some function symbolic(var, step) that creates a symbolic representation of variable at step n
# for example: since the first instruction is `inp w`, f(w, 1) will be i_0
# since the second instruction is `mul x 0`, f(x, 2) will be 0.
# the third expression is `add x z`, so f(x, 3) will be 0, etcetera

def apply_aritmetic_symbolic(instr, a, b):
    # a and b are strings
    if is_const(a) and is_const(b): return str(apply_aritmetic(instr, int(a), int(b)))
    if instr == 'add' and a == 0: return b
    if instr == 'add' and b == 0: return a
    if instr == 'add': return f"({a}+{b})"
    if instr == 'mul' and b == 0: return '0'
    if instr == 'mul' and a == 0: return '0'
    if instr == 'mul': return f"({a}*{b})"
    if instr == 'div' and b == 1: return a
    if instr == 'div': return f"({a}/{b})"
    if instr == 'mod': return f"({a}%{b})"
    if instr == 'eql': return f"({a}={b})"
    raise ValueError(f"{instr} is not a correct aritmetic operation")


def apply_instruction_symbolic(instruction, state, model_number):
    if instruction.startswith('inp'):
        return set_value(state, instruction[-1], model_number[0]), model_number[1:]
    instr, var_a, var_b = instruction.split(' ')
    const_a, const_b = get_value(state, var_a), get_value(state, var_b)
    #print(state)
    state = set_value(state, var_a, apply_aritmetic_symbolic(instr, const_a, const_b))
    return state, model_number

stepn = 1
state_library = {0: ['0'] * 4}
model_number = [f'm{i}' for i in range(14)]

for stepn in range(20):# range(len(data)):
    instr = data[stepn]
    state_library[stepn+1], model_number = apply_instruction_symbolic(instr, state_library[stepn], model_number)
    print(state_library[stepn+1])

#with open('24.tmp', 'w') as f:
#    f.write(str(state_library[252][3]))
# The output is state_library[252][3], which is about 80MB written to disk... damn


['m0', '0', '0', '0']
['m0', '0', '0', '0']
['m0', '0', '0', '0']
['m0', '0', '0', '0']
['m0', '0', '0', '0']
['m0', '11', '0', '0']
['m0', '(11=m0)', '0', '0']
['m0', '((11=m0)=0)', '0', '0']
['m0', '((11=m0)=0)', '0', '0']
['m0', '((11=m0)=0)', '25', '0']
['m0', '((11=m0)=0)', '(25*((11=m0)=0))', '0']
['m0', '((11=m0)=0)', '((25*((11=m0)=0))+1)', '0']
['m0', '((11=m0)=0)', '((25*((11=m0)=0))+1)', '0']
['m0', '((11=m0)=0)', '0', '0']
['m0', '((11=m0)=0)', 'm0', '0']
['m0', '((11=m0)=0)', '(m0+8)', '0']
['m0', '((11=m0)=0)', '((m0+8)*((11=m0)=0))', '0']
['m0', '((11=m0)=0)', '((m0+8)*((11=m0)=0))', '((m0+8)*((11=m0)=0))']
['m1', '((11=m0)=0)', '((m0+8)*((11=m0)=0))', '((m0+8)*((11=m0)=0))']
['m1', '0', '((m0+8)*((11=m0)=0))', '((m0+8)*((11=m0)=0))']


In [24]:
# That ... didn't really work...
# Next approach: getting a tip on reddit.
# I found advice: see if there is something common done to all/most input digits, and if you can use that to constrain those digits
# The rest I all figured out without help

# The common factor in a lot of the input is:
# imp w
# mul x 0
# add x z
# mod x 26
# div z V1 (26 or 1)
# add x V2
# eql x w
# eql x 0
# mul y 0
# add y 25
# mul y x
# add y 1
# mul z y
# mul y 0
# add y w
# add y V3
# mul y x
# add z y

# So let's detect this and reduce it!

def pattern_detector(instructions, start):
    if all([i.startswith(j) for (i, j) in zip(
        instructions[start:(start+18)],
        ['inp', 'mul', 'add', 'mod', 'div', 'add', 'eql', 'eql', 'mul', 'add', 'mul', 'add', 'mul', 'mul', 'add', 'add', 'mul', 'add']
        )]):
        return 1
    return 0

sum(pattern_detector(data, i) for i in range(250))
# Looks like this pattern happens to all 14 digits. also the input size is 14 * 18 = 252 so it matches up

14

In [25]:
for i in range(14):
    var1 = data[18*i + 4].split(' ')[2]
    var2 = data[18*i + 5].split(' ')[2]
    var3 = data[18*i + 15].split(' ')[2]
    print(f"OPERATION({var1}, {var2}, {var3})")

OPERATION(1, 11, 8)
OPERATION(1, 14, 13)
OPERATION(1, 10, 2)
OPERATION(26, 0, 7)
OPERATION(1, 12, 11)
OPERATION(1, 12, 4)
OPERATION(1, 12, 13)
OPERATION(26, -8, 13)
OPERATION(26, -9, 10)
OPERATION(1, 11, 1)
OPERATION(26, 0, 2)
OPERATION(26, -5, 14)
OPERATION(26, -6, 6)
OPERATION(26, -12, 14)


In [28]:
stepn = 1
state_library = {0: ['w', 'x', 'y', 'z']}
model_number = ['w']

for stepn in range(18):# range(len(data)):
    instr = data[stepn]
    state_library[stepn+1], model_number = apply_instruction_symbolic(instr, state_library[stepn], model_number)

# Also let's check out what the operation actually does:
state_library[18]

# It looks like it sets:
# w = (input variable)
# x_new = ((((z%26)+V2)=w)=0) ==> simplify ==> (z%26 + V2) != w
# y_new = ((w+8)*((((z%26)+V2)=w)=0)) ==> simplify ==> (w+V3) * x_new
# z_new = (((z/V1)*((25*((((z%26)+V2)=w)=0))+1))+((w+V3)*((((z%26)+V2)=w)=0))) ==> simplify ==> (z/V1)*(25*x_new + 1) + y_new

# Importantly, x_new is always 0 or 1, which means that we can split up on: (x_new is 0 when it equals, 1 when it not equals)
# x_new is 0 -> z = z/V1
# x_new is 1 -> z = (z/V1)*26 + (w + V3)

# Also, V3 is always less than 15, so w+V3 will never exceed 25.
# I also noticed that V1 is 1 7 times, 26 7 times.
# if x_new is always 0, then we should trivially end up with 0 as the output, since z_new = 0/V1, will stay at 0
# however, that is impossible, e.g, at first: 0 % 26 + 11 will never be equal to w
# This leads me to suspect that x_new should be 0 if V1 == 26, and 1 if V1 == 1, so let's work with that

['w',
 '((((z%26)+11)=w)=0)',
 '((w+8)*((((z%26)+11)=w)=0))',
 '((z*((25*((((z%26)+11)=w)=0))+1))+((w+8)*((((z%26)+11)=w)=0)))']

In [29]:
# This assumption implies the restraints: if we can simply ensure x_new is 0 for those steps where V1 == 26, we are done
# Let's look at a much simpler test example:

# OPERATION(1, 10, 2)
# OPERATION(26, 0, 7)

# After the first step, z will be w_0 + 2
# to ensure x_new is 0: w_1 == (w_0 + 2) + 0

# It seems the conditions will 'match' up on 'bit'(26wise) location, so we can match up the constraints:

# OPERATION(1, 11, 8)  - w0, 0
# OPERATION(1, 14, 13) - w1, 1
# OPERATION(1, 10, 2)  - w2, 2
# OPERATION(26, 0, 7)  - w3, 2
# OPERATION(1, 12, 11) - w4, 3
# OPERATION(1, 12, 4)  - w5, 4
# OPERATION(1, 12, 13) - w6, 5
# OPERATION(26, -8, 13)- w7, 5
# OPERATION(26, -9, 10)- w8, 4
# OPERATION(1, 11, 1)  - w9, 6
# OPERATION(26, 0, 2)  - 10, 6
# OPERATION(26, -5, 14)- 11, 3
# OPERATION(26, -6, 6) - 12, 1
# OPERATION(26, -12, 14)-13, 0

# The constraints that result:
# 0 - w13 == w0 + 8 - 12 = w0 - 4
# 1 - w12 == w1 + 13 - 6 = w1 + 7
# 2 - w3 == w2 + 2 - 0 = w2 + 2
# 3 - w11 == w4 + 11 - 5 = w4 + 6
# 4 - w8 == w5 + 4 - 9   = w5 - 5
# 5 - w7 == w6 + 13 - 8  = w6 + 5
# 6 - w10 == w9 + 1 + 0  = w9 + 1

# An example would be:
apply_instructions_list(data, [5, 1, 1, 3, 1, 6, 1, 6, 1, 1, 2, 7, 8, 1])
# Success!

[1, 0, 0, 0]

In [32]:
# Highest possible values (simply because out of every pair, the highest one must be 9)
# w0 = 9, w13 = 5
# w1 = 2, w12 = 9
# w2 = 7, w3 = 9
# w4 = 3, w11 = 9
# w5 = 9, w8 = 4
# w6 = 4, w7 = 9
# w9 = 8, w10 = 9

# gives us
# 92793949489995
apply_instructions_list(data, [9, 2, 7, 9, 3, 9, 4, 9, 4, 8, 9, 9, 9, 5])

[5, 0, 0, 0]

In [33]:
# lowest possible values (simply because out of every pair, the lowest one must be 1)
# w0 = 5, w13 = 1
# w1 = 1, w12 = 8
# w2 = 1, w3 = 3
# w4 = 1, w11 = 7
# w5 = 6, w8 = 1
# w6 = 1, w7 = 6
# w9 = 1, w10 = 2

# gives us
# 51131616112781
apply_instructions_list(data, [5, 1, 1, 3, 1, 6, 1, 6, 1, 1, 2, 7, 8, 1])

[1, 0, 0, 0]